<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_groupchat_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Group Chat with Retrieval Augmented Generation

AutoGen supports conversable agents powered by LLMs, tools, or humans, performing tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install "pyautogen[retrievechat]>=0.2.3"
```

In [3]:
%%capture --no-stderr
# %pip install "pyautogen[retrievechat]>=0.2.3"

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [4]:
import chromadb

import autogen
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-3.5-turbo", "gpt-35-turbo", "gpt-35-turbo-0613", "gpt-4", "gpt4", "gpt-4-32k"],
    },
)

print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))])

LLM models:  ['gpt-4', 'gpt-4-32k']


It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well).

The config list looks like the following:
```python
config_list = [
    {
        "model": "gpt-4",
        "api_key": "<your OpenAI API key>",
    },  # OpenAI API endpoint for gpt-4
    {
        "model": "gpt-35-turbo-0631",  # 0631 or newer is needed to use functions
        "base_url": "<your Azure OpenAI API base>", 
        "api_type": "azure", 
        "api_version": "2023-08-01-preview", # 2023-07-01-preview or newer is needed to use functions
        "api_key": "<your Azure OpenAI API key>"
    }
]
```

You can set the value of config_list in any way you prefer. Please refer to this [notebook](https://github.com/microsoft/autogen/blob/main/notebook/oai_openai_utils.ipynb) for full code examples of the different methods.

## Construct Agents

In [14]:
llm_config = {
    "timeout": 60,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

# autogen.ChatCompletion.start_logging()


def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()


user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    human_input_mode="ALWAYS",
    code_execution_config=False,  # we don't want to execute code in this case.
)

planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an Financial_Analyst_Summarizer who can write financial news summary and a Database_Information_Retriever who can retrieve the latest news articles for give topic from the database, a Content_Review_Scorer who can assess the accuracy and relevance of information shared in the chat, a Summary_Improvement_Advisor who can suggest improvements to the summary, and a Final_Summary_Validator who can validate the final summary.
Explain the plan first. Be clear which step is performed by an Database_Information_Retriever, and which step is performed by a Financial_Analyst_Summarizer.
""",
    llm_config=llm_config,
)

Financial_Analyst_Summarizer = AssistantAgent(
    name="Financial_Analyst_Summarizer",
    llm_config=llm_config,
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    system_message="You are now in a group chat. You need to complete a task with other participants. As a Financial Analyst Summarizer, your role involves generating concise and informative summaries of news articles on given financial topics. Once the summary is created, you will review it, score it based on its accuracy and relevance, and provide detailed feedback for improvement. When necessary, you may retrieve additional information from the database to enhance the summary further. After incorporating the improvements, you will present the updated summary back to the author for final feedback. You must also be prepared to provide a final score that reflects the quality of the final summary, ensuring your peers receive the most critical information effectively and efficiently.\n\nIn your role, you have the autonomy to question the content or the process presented in this group chat and can request corrections or seek clarification if there is something that appears to be missing or unclear after executing a given task. If at any point you find yourself confused or in need of assistance, do not hesitate to reach out to the group chat manager, who can guide you or delegate the task to another qualified participant. Your proactive communication and feedback will ensure the delivery of high-quality, timely financial news summaries to your team.\n\nRemember, your expertise as a financial analyst is paramount in ensuring that the information provided is relevant and up-to-date for important financial decisions. The task is considered complete when you believe the summary meets the standards set forth and has fulfilled the informational needs of your audience, at which point you should reply `TERMINATE` to signal the end of the task.",
    code_execution_config=False,  # we don't want to execute code in this case.
    default_auto_reply="Reply `TERMINATE` if the task is done.",
    description="Financial_Analyst_Summarizer is a role requiring strong financial analysis acumen, the ability to comprehend and summarize complex financial data, and excellent communication skills to clarify financial concepts and correct misunderstandings within group chat discussions. They should be detail-oriented, possess critical thinking ability to question and verify the accuracy of financial statements or forecasts, and offer precise corrections when needed. Although not primarily a programming role, proficiency in Python may be beneficial for automating financial data analysis and providing timely insights in the group chat.",
)

Content_Review_Scorer = AssistantAgent(
    name="Content_Review_Scorer",
    llm_config=llm_config,
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    system_message="You are now in a group chat where your primary role as a Content_Review_Scorer is to generate a summary of financial news articles, review the produced summaries, score them, and provide feedback for improvement. As a financial analyst preparing crucial summaries for your colleagues, you must ensure accuracy and comprehensiveness in the conveyed information. Your task involves:\n\n- Carefully reading and understanding the thematic content of news articles in the financial sector.\n- Creating clear and concise summaries that capture the key points and essential data.\n- Reviewing and critically assessing the quality of these summaries, assigning an appropriate score based on predefined criteria such as relevance, accuracy, completeness, and clarity.\n- Offering constructive feedback aimed at enhancing the summarizer's work, highlighting areas of the summary that may require additional information or could benefit from refinement.\n- When necessary, retrieving missing information from a database to supplement and improve the summary's content, ensuring that all relevant details are included.\n- After the summarizer has integrated your feedback and revised the summary, performing a final review to confirm that all improvements have been effectively made.\n- Providing a final score for the improved summary, reflecting any enhancements made as a result of your feedback.\n\nYour engagement in this task is iterative, and you will depend on your expertise in financial analysis to make judgements about the content. If at any point you are unsure or need clarification, you are encouraged to seek assistance from the group chat manager, who will facilitate further discussion or select another participant to provide support.\n\nUltimately, once you are satisfied that the summary is of high quality and meets the requirements, and once all your colleagues in the chat are content with the resolved issues, you or another participant may conclude the session with `TERMINATE.` \n\nThis role does not involve coding; instead, you will be leveraging your financial expertise and communication skills to accomplish these tasks. If you encounter any issues, such as a lack of output after an information retrieval attempt, feel free to voice your concerns and work collaboratively to correct any oversights.",
    code_execution_config=False,  # we don't want to execute code in this case.
    default_auto_reply="Reply `TERMINATE` if the task is done.",
    description="The Content_Review_Scorer is a detail-oriented individual with strong analytical skills and a good understanding of content quality standards, who can assess the accuracy and relevance of information shared in the chat. They should possess excellent communication skills to provide constructive feedback and suggest improvements, and have the ability to read and comprehend Python code to evaluate its execution and outcomes effectively. The holder of this position does not necessarily develop code but should be proficient enough in Python to identify issues and propose solutions when necessary.",
)

Summary_Improvement_Advisor = AssistantAgent(
    name="Summary_Improvement_Advisor",
    llm_config=llm_config,
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    system_message="You are now in a group chat tasked with the critical role of Summary_Improvement_Advisor for financial analysts. As an advanced AI assistant, you possess the expertise to assess and enhance financial summaries using your language skills. Although python coding isn't within your scope, you should be capable of extracting pertinent information from various sources without the need for scripts.\n\nYour role involves meticulously reviewing summaries created by a financial analyst, assigning them a score based on accuracy, coherence, and completeness, and offering constructive feedback. Whenever a summary lacks certain details or could be improved, you will retrieve the necessary information, likely from financial databases, and refine the content aspiring for excellence.\n\nWhen facing challenges or if the task appears to be at a standstill, do not hesitate to seek guidance from the group chat manager, who has the authority to appoint another participant to assist or to take over the task as needed. Keep in mind the collaborative nature of this environment; you have the autonomy to question any previous messages or work presented in the group chat and suggest appropriate amendments.\n\nYour objective is to work with financial analysts to ensure the final summary is of superior quality, reflecting the most significant aspects of the financial topics at hand. The overall success is marked by both the analyst's satisfaction with your improvements and the final score you attribute to the summary.\n\nWhen you believe that the task has been satisfactorily completed—that is, the final summary is polished, and all feedback has been addressed—you should conclude the interaction with `TERMINATE`.",
    code_execution_config=False,  # we don't want to execute code in this case.
    default_auto_reply="Reply `TERMINATE` if the task is done.",
    description="The Summary_Improvement_Advisor is a critical thinker with strong analytical skills and expertise in Python, capable of scrutinizing previous discussions or Python code snippets shared in the group chat. This individual must excel in clear communication to provide constructive feedback and alternative solutions that enhance the group's understanding and productivity. They should be encouraged to speak up whenever they spot inaccuracies or areas for improvement to ensure optimal group outcomes.",
)

Final_Summary_Validator = AssistantAgent(
    name="Final_Summary_Validator",
    llm_config=llm_config,
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    system_message="You are now in a group chat. You need to complete a task with other participants. As a Final_Summary_Validator, your role is to critically evaluate and enhance summaries of financial news articles created by a financial analyst for an internal audience. \n\nYour task involves:\n\n1. Generating a summary of the news articles on a given financial topic using your analytical skill.\n2. Reviewing this summary carefully, assigning it a score based on its accuracy, coverage, and readability, and providing constructive feedback to the summarizer.\n3. Retrieving any missing or additional pertinent information from a database to refine the summary without using code. You are expected to have the necessary access and authorization to handle relevant data securely.\n4. Improving the summary based on the retrieved information and re-evaluating it, ensuring that the final product is comprehensive and up to the expected standard.\n5. Providing the author with a re-scored and revised summary and awaiting their feedback.\n6. Once the author's feedback is received, you will provide a final summary and its final score.\n\nRemember, your role does not involve coding. Should you encounter difficulties or uncertainties while executing your tasks, do not hesitate to ask for help from the group chat manager. They will guide you or redirect you to another team member. \n\nIn case of doubts about previous messages, such as a lack of output after an analyst has claimed to improve the summary, you should voice your concerns and contribute a corrected approach.\n\nEnd the task by stating `TERMINATE` in the group chat when you have determined that the task requirements have been fully satisfied.",
    code_execution_config=False,  # we don't want to execute code in this case.
    default_auto_reply="Reply `TERMINATE` if the task is done.",
    description="Final_Summary_Validator is a critical thinker and detail-oriented individual who possesses strong analytical skills and a good understanding of Python coding. This role is responsible for scrutinizing previous messages and code within the group chat, identifying any inaccuracies or errors, and providing a corrected summary or revised Python code when necessary. The manager should allow the Final_Summary_Validator to interject when clarity and accuracy are paramount, especially following discussions that have led to potentially flawed conclusions or code without expected outputs.",
)

Database_Information_Retriever = RetrieveUserProxyAgent(
    name="Content_Review_Scorer",
    # system_message="You are now in a group chat where your primary role as a Content_Review_Scorer is to generate a summary of financial news articles, review the produced summaries, score them, and provide feedback for improvement. As a financial analyst preparing crucial summaries for your colleagues, you must ensure accuracy and comprehensiveness in the conveyed information. Your task involves:\n\n- Carefully reading and understanding the thematic content of news articles in the financial sector.\n- Creating clear and concise summaries that capture the key points and essential data.\n- Reviewing and critically assessing the quality of these summaries, assigning an appropriate score based on predefined criteria such as relevance, accuracy, completeness, and clarity.\n- Offering constructive feedback aimed at enhancing the summarizer's work, highlighting areas of the summary that may require additional information or could benefit from refinement.\n- When necessary, retrieving missing information from a database to supplement and improve the summary's content, ensuring that all relevant details are included.\n- After the summarizer has integrated your feedback and revised the summary, performing a final review to confirm that all improvements have been effectively made.\n- Providing a final score for the improved summary, reflecting any enhancements made as a result of your feedback.\n\nYour engagement in this task is iterative, and you will depend on your expertise in financial analysis to make judgements about the content. If at any point you are unsure or need clarification, you are encouraged to seek assistance from the group chat manager, who will facilitate further discussion or select another participant to provide support.\n\nUltimately, once you are satisfied that the summary is of high quality and meets the requirements, and once all your colleagues in the chat are content with the resolved issues, you or another participant may conclude the session with `TERMINATE.` \n\nThis role does not involve coding; instead, you will be leveraging your financial expertise and communication skills to accomplish these tasks. If you encounter any issues, such as a lack of output after an information retrieval attempt, feel free to voice your concerns and work collaboratively to correct any oversights.",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "qa",
        "docs_path": None,
        "model": config_list[0]["model"],
        "client": chromadb.PersistentClient(path="/workspaces/autogen/usecases/nb/chroma_reuters21578"),
        "collection_name": "test",
        "get_or_create": True,
    },
    code_execution_config=False,  # we don't want to execute code in this case.
)


In [15]:

PROBLEM = "Agip Petroli"


def _reset_agents():
    user_proxy.reset()
    Financial_Analyst_Summarizer.reset()
    Content_Review_Scorer.reset()
    Summary_Improvement_Advisor.reset()
    Final_Summary_Validator.reset()
    Database_Information_Retriever.reset()

In [16]:
from typing_extensions import Annotated

def retrieve_content(
        message: Annotated[
            str,
            "Refined message which keeps the original meaning and can be used to retrieve content for code generation and question answering.",
        ],
        n_results: Annotated[int, "number of results"] = 3,
    ) -> str:
        Database_Information_Retriever.n_results = n_results  # Set the number of results to be retrieved.
        # Check if we need to update the context.
        update_context_case1, update_context_case2 = Database_Information_Retriever._check_update_context(message)
        if (update_context_case1 or update_context_case2) and Database_Information_Retriever.update_context:
            Database_Information_Retriever.problem = message if not hasattr(Database_Information_Retriever, "problem") else Database_Information_Retriever.problem
            _, ret_msg = Database_Information_Retriever._generate_retrieve_user_reply(message)
        else:
            _context = {"problem": message, "n_results": n_results}
            ret_msg = Database_Information_Retriever.message_generator(Database_Information_Retriever, None, _context)
        return ret_msg if ret_msg else message

In [17]:

Database_Information_Retriever.human_input_mode = "NEVER"  # Disable human input for Database_Information_Retriever since it only retrieves content.
agents_lst = [Financial_Analyst_Summarizer, Content_Review_Scorer, Summary_Improvement_Advisor, Final_Summary_Validator]

for caller in agents_lst:
    d_retrieve_content = caller.register_for_llm(
        description="retrieve content for code generation and question answering.", api_style="function"
    )(retrieve_content)

for executor in [user_proxy]:
    executor.register_for_execution()(d_retrieve_content)

groupchat = autogen.GroupChat(
    agents=[user_proxy, Financial_Analyst_Summarizer, Content_Review_Scorer, Summary_Improvement_Advisor, Final_Summary_Validator],
    messages=[],
    max_round=12,
    speaker_selection_method="round_robin",
    allow_repeat_speaker=False,
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=
                                   {
                                        "timeout": 60,
                                        "temperature": 0,
                                        "config_list": config_list,
                                    }
                                   )

# Start chatting with the boss as this is the user proxy agent.
user_proxy.initiate_chat(
    manager,
    message=PROBLEM,
)


Admin (to chat_manager):

Agip Petroli

--------------------------------------------------------------------------------
Financial_Analyst_Summarizer (to chat_manager):

***** Suggested function Call: retrieve_content *****
Arguments: 
{"message":"Agip Petroli latest financial news"}
*****************************************************

--------------------------------------------------------------------------------
Admin (to chat_manager):

Admin (to chat_manager):

***** Response from calling function "retrieve_content" *****
USER INTERRUPTED
*************************************************************

--------------------------------------------------------------------------------
first retrieve info using retrieve_content function call

--------------------------------------------------------------------------------
Financial_Analyst_Summarizer (to chat_manager):

***** Suggested function Call: retrieve_content *****
Arguments: 
{"message":"Agip Petroli latest financial news"}
*

ChatResult(chat_id=None, chat_history=[{'content': 'Agip Petroli', 'role': 'assistant'}, {'content': '', 'function_call': {'arguments': '{"message":"Agip Petroli latest financial news"}', 'name': 'retrieve_content'}, 'name': 'Financial_Analyst_Summarizer', 'role': 'assistant'}, {'content': 'first retrieve info using retrieve_content function call', 'tool_responses': [{'role': 'function', 'name': 'retrieve_content', 'content': 'USER INTERRUPTED'}], 'role': 'assistant'}, {'content': '', 'function_call': {'arguments': '{"message":"Agip Petroli latest financial news"}', 'name': 'retrieve_content'}, 'name': 'Financial_Analyst_Summarizer', 'role': 'assistant'}, {'content': "You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the\ncontext provided by the user.\nIf you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.\nYou must give as short an answer as possible.\n\nUser's question is: Agip Pe

In [ ]:
# Start chatting with Database_Information_Retriever as this is the user proxy agent.
user_proxy.initiate_chat(
    manager,
    problem=PROBLEM,
    n_results=3,
)

## Start Chat

### UserProxyAgent doesn't get the correct code
[FLAML](https://github.com/microsoft/FLAML) was open sourced in 2020, so ChatGPT is familiar with it. However, Spark-related APIs were added in 2022, so they were not in ChatGPT's training data. As a result, we end up with invalid code.

In [ ]:
norag_chat()

Boss (to chat_manager):

what is ifrs?

--------------------------------------------------------------------------------


Product_Manager (to chat_manager):

IFRS stands for International Financial Reporting Standards. It is a set of accounting standards developed by the International Accounting Standards Board (IASB), which is an independent, not-for-profit organization. The goal of IFRS is to provide a global framework for how public companies prepare and disclose their financial statements, ensuring transparency, accountability, and efficiency in the world's financial markets.

By adopting IFRS, companies make their financial statements understandable and comparable across international boundaries. This is particularly beneficial for companies that have operations in multiple countries or are seeking to attract international investors. IFRS covers a wide range of accounting activities, including revenue recognition, financial statement presentation, and the reporting of assets and liabilities.

As of my knowledge cutoff date in 2023, more than 140 countries either require or permit the use of IFRS. How

### RetrieveUserProxyAgent get the correct code
Since RetrieveUserProxyAgent can perform retrieval-augmented generation based on the given documentation file, ChatGPT can generate the correct code for us!

In [ ]:
rag_chat()
# type exit to terminate the chat

Boss_Assistant (to chat_manager):

what is sppi

--------------------------------------------------------------------------------
Senior_Python_Engineer (to chat_manager):

It seems like there might be a typo or misunderstanding in your query. "SPPI" could refer to a variety of things depending on the context, but without additional information, it's difficult to provide a precise answer. Here are a few possibilities:

1. **SPPI (Simplified Payment Verification Protocol for Invoices)**: This could be a reference to a protocol related to blockchain or cryptocurrency transactions, although the more common term is SPV (Simplified Payment Verification) which is used in Bitcoin to verify transactions without downloading the entire blockchain.

2. **SPPI (Sole Purpose of Payment Indicator)**: In financial contexts, this might refer to a code or indicator used in payment systems to specify the purpose of a transaction.

3. **SPPI (Standardized Precipitation and Evapotranspiration Index)**: In

### Call RetrieveUserProxyAgent while init chat with another user proxy agent
Sometimes, there might be a need to use RetrieveUserProxyAgent in group chat without initializing the chat with it. In such scenarios, it becomes essential to create a function that wraps the RAG agents and allows them to be called from other agents.

In [ ]:
_reset_agents()

Database_Information_Retriever.retrieve_docs("what is sppi", 3 )



doc_ids:  [['10442f02-534f-4697-8e2e-75a3bc9fda2e', 'c641a6b3-735f-4090-8e76-15f1b8e23802', '205cf24c-9d2f-47a6-9546-d024ea005cec']]


In [ ]:
call_rag_chat()

Boss (to chat_manager):

what is ifrs?

--------------------------------------------------------------------------------
Code_Reviewer (to chat_manager):

IFRS stands for International Financial Reporting Standards. These are a set of accounting standards developed by the International Accounting Standards Board (IASB), which is an independent, not-for-profit organization. IFRS provides a global framework for how public companies prepare and disclose their financial statements, giving investors and other users of financial information the ability to compare the financial performance of publicly listed companies on a like-for-like basis with their international peers.

IFRS standards are designed to be applied by companies on a consistent basis throughout the world, which helps to ensure transparency, accountability, and efficiency in financial markets globally. This is particularly important for companies that operate in multiple countries or that have investors from different countrie